In [1]:
import sys
sys.path.append("/home/sunhokim/Documents/mygit/TrackAndRe-ID")
sys.path.append("/home/sunhokim/Documents/mygit/TrackAndRe-ID/mylibrary")
import os
from os.path import isfile, join

import cv2
import numpy as np

import torchreid

## Buf Loader

In [2]:
def list_images_in_directory(directory_path):
    """
    List all image files in the specified directory.

    :param directory_path: The path to the directory containing images.
    :return: A list of image file paths.
    """
    image_files = [join(directory_path, f) for f in os.listdir(directory_path) if isfile(join(directory_path, f))]

    # Filter the image files based on common image file extensions (you can customize this list).
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']
    image_files = [f for f in image_files if any(f.lower().endswith(ext) for ext in image_extensions)]

    return image_files

def load_image(file_path):
    # Function to load an image from the specified file_path.
    # You can load the image in grayscale using cv2.IMREAD_GRAYSCALE or in color using cv2.IMREAD_COLOR.
    image = cv2.imread(file_path, cv2.IMREAD_COLOR)

    if image is None:
        # If the image cannot be loaded, print an error message.
        print(f"Unable to load the image: {file_path}")
        return None

    # Convert the image from BGR color format (used by OpenCV) to RGB color format.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    return image

In [26]:
# def split_list_into_batches(input_list, batch_size):
#     """
#     Split a list into batches of a specified size.

#     :param input_list: The input list to be split.
#     :param batch_size: The size of each batch.
#     :return: A list of batches, where each batch is a list.
#     """
#     batched_list = []
#     for i in range(0, len(input_list), batch_size):
#         batched_list.append(input_list[i:i+batch_size])
#     return batched_list

# import os
import re

def split_list_into_batches(image_list):
    """
    Split a list of image files into batches based on 'id{index}_' part of the file names.
    
    :param image_list: A list of image file paths.
    :return: A dictionary where keys are 'index' values, and values are lists of file paths with the same 'id{index}_'.
    """
    batches = {}  # Dictionary to store batches
    for image_path in image_list:
        # Use regular expression to extract 'index' from the file name
        match = re.search(r'id(\d+)_', image_path)
        if match:
            index = int(match.group(1))  # Extract the 'index'
            if index in batches:
                batches[index].append(image_path)
            else:
                batches[index] = [image_path]
    
    return batches


## Feature Extractor

In [27]:
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_ain_x1_0',
    model_path='weights/reid/imageNet-market.pth.tar-60',
    device='cuda'
)

directory_path = '/home/sunhokim/Documents/mygit/TrackAndRe-ID/images/buf-s/0'
image_list = list_images_in_directory(directory_path)
batches = split_list_into_batches(image_list)

features = {}
for key, value in batches.items():
    features[key]=extractor(value)

/home/sunhokim/Documents/mygit/TrackAndRe-ID/torchreid/torchreid/utils/tools.py:43: UserWarning: No file found at "weights/reid/imageNet-market.pth.tar-60"
  warnings.warn('No file found at "{}"'.format(fpath))


Successfully loaded imagenet pretrained weights from "/home/sunhokim/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_ain_x1_0
- params: 2,193,616
- flops: 978,878,352


In [30]:
print(len(features)) # output (?, 512)
print(features.keys())
features[35].size()

33
dict_keys([35, 54, 82, 63, 66, 40, 26, 107, 93, 65, 80, 64, 95, 30, 55, 127, 52, 105, 133, 114, 50, 100, 14, 134, 44, 129, 13, 53, 18, 32, 31, 124, 120])


torch.Size([199, 512])

In [37]:
from torchreid import metrics
import torch

dismat = metrics.compute_distance_matrix(features[82], features[134], metric='cosine')
print(dismat.size())
print(torch.max(dismat).item())
print(torch.min(dismat).item())

torch.Size([97, 20])
0.3166714906692505
0.05854642391204834


minimum distance가 .1 미만이면 합쳐도 될듯?